In [9]:
import pygplates
import numpy as np
from mpl_toolkits.basemap import Basemap
from topology_plotting import *
from matplotlib.patches import Polygon
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

# Specify some input files

#rotation_filename = '/Users/Simon/GIT/pygplates-alpha/tutorials-sv/Data/Seton_etal_ESR2012_2012.1.rot'
#input_topology_filename = '../tutorials-sv/Data/Seton_etal_ESR2012_PP_2012.1.gpmlz'
#topology_features = pygplates.FeatureCollection(input_topology_filename)


GPML_List = ['./MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',\
             './MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']

topology_features = pygplates.FeatureCollection()
for file in GPML_List:
    topology_feature = pygplates.FeatureCollection(file)
    topology_features.add(topology_feature)

#pygplates.FeatureCollection(continental_polygon_features).write('./MullerData/continental_polygons.gpmlz')
        
# End of input specification

###########################################################

coastlines_file = './MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_Coastlines.gpmlz'



In [12]:
# Run analysis for subduction initiation

rotation_filename = './MullerData/Global_EarthByte_230-0Ma_GK07_AREPS.rot'
rotation_model = pygplates.RotationModel(rotation_filename)
static_polygon_filename = './MullerData/continental_polygons.gpmlz'


# Time range and time increment for convergence velocity calculation
time_step = 1.
time_offset = 1.
time_start = 0
time_end = 230.

results_initiation = []

highlighting = 'init_death'

# Create an empty dataframe to concatenate results to
df_init = pd.read_csv('./results/subduction_initiation_230_0.csv')
df_death = pd.read_csv('./results/subduction_death_230_0.csv')

for time in np.arange(time_start,time_end,time_step):
    
    #szdata=[]
    #for item in zip(szLon,szLat,dist,dist2cont,sAge):
    #    szdata.append(item)
    #df = pd.DataFrame(szdata, columns = DataFrameTemplate)
    
    #df_AllTimes = df_AllTimes.append(df)
    
    df_init_t = df_init[(df_init['Age']==time)]
    subset_init = df_init_t[(df_init_t['dist2sz']<-200.)]
    
    df_death_t = df_death[(df_death['Age']==time)]
    subset_death = df_death_t[(df_death_t['dist2sz']<-200.)]
    
    
    # Plotting
    pygplates.reconstruct(coastlines_file, rotation_model, 'tmp.shp', time)

    pygplates.reconstruct(continental_polygon_features, rotation_model, 'tmp2.shp', time)
    
    fig = plt.figure(figsize=(10,5),dpi=150)
    ax_map = fig.add_axes([0.01,0.01,0.9,0.98])
    lon0=0
    m = Basemap(projection='moll', lon_0=lon0, resolution='c', ax=ax_map)
    cp = m.drawmapboundary()
    m.drawparallels(np.arange(-90,90,30))
    m.drawmeridians(np.arange(-180,180,30))

    # Plot reconstructed coastlines
    shp_info = m.readshapefile('tmp','shp',drawbounds=True,color='none')
    for nshape,seg in enumerate(m.shp):
        poly = Polygon(seg,facecolor='DarkKhaki',edgecolor='none',alpha=0.5,zorder=0.5)
        plt.gca().add_patch(poly)
        
    # Plot reconstructed coastlines
    shp_info = m.readshapefile('tmp2','shp',drawbounds=True,color='none')
    for nshape,seg in enumerate(m.shp):
        poly = Polygon(seg,facecolor='grey',edgecolor='none',alpha=0.3,zorder=0.25)
        plt.gca().add_patch(poly)

    x,y = m(np.array(df_init_t['lon']),np.array(df_init_t['lat']))
    l1 = m.scatter(x,y,c=np.array(df_init_t['dist2cont']),s=10,
            edgecolors='',zorder=2,cmap=plt.cm.plasma_r,vmin=0,vmax=2000)
    #l1 = m.plot(x,y,'.',color='gray',zorder=1,alpha=0.2)
    
    if 'init' in highlighting:
        x,y = m(np.array(subset_init['lon']),np.array(subset_init['lat']))
        m.plot(x,y,'o',color='steelblue',markersize=13)
        l1 = m.scatter(x,y,c=np.array(subset_init['dist2cont']),s=70,
                edgecolors='',zorder=3,cmap=plt.cm.plasma_r,vmin=0,vmax=2000)
    
    if 'death' in highlighting:
        x,y = m(np.array(subset_death['lon']),np.array(subset_death['lat']))
        m.plot(x,y,'o',color='black',markersize=13)
        l1 = m.scatter(x,y,c=np.array(subset_death['dist2cont']),s=70,
                edgecolors='',zorder=3,cmap=plt.cm.plasma_r,vmin=0,vmax=2000)
    
    if highlighting=='init':
        plt.title('Subduction at %s Ma | Initiation outlined in Blue | Colouring by distance to continent' % time, size=11)
    if highlighting=='death':
        plt.title('Subduction at %s Ma | Cessation outlined in Black | Colouring by distance to continent' % time, size=11)
    if highlighting=='init_death':
        plt.title('Subduction at %s Ma | Initiation outlined in Blue | Cessation outlined in Black | Colouring by distance to continent' % time, size=11)
    
    divider = make_axes_locatable(ax_map)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    plt.colorbar(l1,extend='max',label='distance (km)',cax=cax)
    
    #plt.show()
    
    plt.savefig('./images/sub_%s_%sMa.png' % (highlighting,time))
    plt.close()
    

In [15]:

rotation_model = pygplates.RotationModel('./MullerData/Global_EarthByte_230-0Ma_GK07_AREPS_VDM.rot')
static_polygon_filename = './MullerData/continental_polygons.gpmlz'

df_2012 = pd.read_csv('./results/VDM_subduction_distances.csv')    

df_2009 = pd.read_csv('./results/VDM_subduction_distances_2009list.csv')  

df_init = pd.read_csv('./results/VDM_subduction_initiation.csv')

df_death = pd.read_csv('./results/VDM_subduction_death.csv')

#df_2012
time_list = df_2012.Age.sort_values().unique()

print time_list

highlighting = 'init'

for time in time_list:
    
    df_2012_t = df_2012[(df_2012['Age']==time)]
    
    df_2009_t = df_2009[(df_2009['Age']==time)]
    
    df_init_t = df_init[(df_init['Age']==time)]
    
    df_death_t = df_death[(df_death['Age']==time)]
    
    rtime=time
    if time==235:
        rtime=231. # hack to workaround the relative rotations only reaching 231 Ma
    print rtime
    
    pygplates.reconstruct(coastlines_file, rotation_model, 'tmp.shp', rtime)

    pygplates.reconstruct(continental_polygon_features, rotation_model, 'tmp2.shp', rtime)
    
    fig = plt.figure(figsize=(10,5),dpi=150)
    ax_map = fig.add_axes([0.01,0.01,0.9,0.98])
    lon0=0
    m = Basemap(projection='moll', lon_0=lon0, resolution='c', ax=ax_map)
    cp = m.drawmapboundary()
    m.drawparallels(np.arange(-90,90,30))
    m.drawmeridians(np.arange(-180,180,30))

    # Plot reconstructed coastlines
    shp_info = m.readshapefile('tmp','shp',drawbounds=True,color='none')
    for nshape,seg in enumerate(m.shp):
        poly = Polygon(seg,facecolor='DarkKhaki',edgecolor='none',alpha=0.5,zorder=0.5)
        plt.gca().add_patch(poly)
        
    # Plot reconstructed coastlines
    shp_info = m.readshapefile('tmp2','shp',drawbounds=True,color='none')
    for nshape,seg in enumerate(m.shp):
        poly = Polygon(seg,facecolor='grey',edgecolor='none',alpha=0.3,zorder=0.25)
        plt.gca().add_patch(poly)

    x,y = m(np.array(df_2012_t['lon']),np.array(df_2012_t['lat']))
    l1 = m.scatter(x,y,c=np.array(df_2012_t['dist2cont']),s=10,
            edgecolors='',zorder=2,cmap=plt.cm.plasma_r,vmin=0,vmax=2000,alpha=0.2)
    
    x,y = m(np.array(df_2009_t['lon']),np.array(df_2009_t['lat']))
    l1 = m.scatter(x,y,c=np.array(df_2009_t['dist2cont']),s=20,
            edgecolors='',zorder=2,cmap=plt.cm.plasma_r,vmin=0,vmax=2000)
    
    if 'init' in highlighting:
        x,y = m(np.array(df_init_t['lon']),np.array(df_init_t['lat']))
        m.plot(x,y,'o',color='steelblue',markersize=13)
        l1 = m.scatter(x,y,c=np.array(df_init_t['dist2cont']),s=70,
                edgecolors='',zorder=3,cmap=plt.cm.plasma_r,vmin=0,vmax=2000)
    
    if 'death' in highlighting:
        x,y = m(np.array(df_death_t['lon']),np.array(df_death_t['lat']))
        m.plot(x,y,'o',color='black',markersize=13)
        l1 = m.scatter(x,y,c=np.array(df_death_t['dist2cont']),s=70,
                edgecolors='',zorder=3,cmap=plt.cm.plasma_r,vmin=0,vmax=2000)

    if highlighting=='init':
        plt.title('Subduction at %s Ma | Initiation outlined in Blue | Colouring by distance to continent' % time, size=11)
    if highlighting=='death':
        plt.title('Subduction at %s Ma | Cessation outlined in Black | Colouring by distance to continent' % time, size=11)
    if highlighting=='init_death':
        plt.title('Subduction at %s Ma | Initiation outlined in Blue | Cessation outlined in Black | Colouring by distance to continent' % time, size=11)
        
    divider = make_axes_locatable(ax_map)
    cax = divider.append_axes("right", size="2%", pad=0.1)
    plt.colorbar(l1,extend='max',label='distance (km)',cax=cax)
    
    #plt.show()
    
    plt.savefig('./images/VDM_sub_%s_%sMa.png' % (highlighting,time))
    plt.close()
    



[  8.  24.  32.  43.  52.  59.  68.  77.  87.  98. 110. 125. 142. 158.
 175. 192. 207. 221. 235.]
8.0
24.0
32.0
43.0
52.0
59.0
68.0
77.0
87.0
98.0
110.0
125.0
142.0
158.0
175.0
192.0
207.0
221.0
231.0
